In [1]:
import os
import openpyxl as openpyxl
import pandas as pd
import numpy as np
import json as json
import datetime

In [23]:
df = pd.DataFrame(pd.read_excel('E:\\PolyU\\gis\\Dissertation\\Data\\深圳市\\2022_7_22_0_3_16\\2022_7_22_0_2_37.xlsx'))
# print(df)
DataFrame = exportUsableData('E:\\PolyU\\gis\\Dissertation\\Data\\深圳市\\2022_7_22_0_3_16\\2022_7_22_0_2_37.xlsx')
# print(DataFrame)
DF = DataFrame[DataFrame['status'] == 3]
DF = DF.loc[:,['id','longitude','latitude']]
# print(DF)
df_copy=DF.drop_duplicates('id')
# print(df_copy)
df_copy = df_copy.reset_index()
df_copy = df_copy.drop(['index'], axis = 1)
print(df_copy)
df_copy.to_csv('E:\\PolyU\\gis\\Dissertation\\Data\\深圳市\\SZstations.csv')

250
266
316
1518
1521
1560
1565
1566
         id   longitude   latitude
0     26403  113.830914  22.720581
1     26282  113.842774  22.720600
2     25208  114.149414  22.602618
3     90519  114.057376  22.522617
4     90532  114.233941  22.632500
..      ...         ...        ...
784  118937  114.126909  22.598395
785  118957  114.146835  22.607674
786  118969  113.912050  22.530845
787  122003  114.142112  22.712706
788   95797  114.352429  22.711030

[789 rows x 3 columns]


In [2]:
def json2dict(rawJson):
    d = json.loads(rawJson)
    return d["data"]

In [3]:
def isPileDetailsListNan(d):
    if len(d["pileDetailsList"]) == 0:
        return True
    return False

In [4]:
def formatCSVname(filename):
    csvname = filename[:-4]
    csvname += "csv"
    return csvname

In [6]:
def listAllExcelPath(cityPath):
    dayPath = os.listdir(cityPath)
    allDayPath = []
    for day in dayPath:
        if 'zip' not in day and 'csv' not in day:
            allDayPath.append(os.path.join(cityPath, day))
    allPath = []
    for curPath in allDayPath:
        allTime = os.listdir(curPath)
        for curTime in allTime:
            if 'xlsx' in curTime :
                if newCSVFileName(curTime, "_charging") not in allTime or newCSVFileName(curTime, "_chargingNo") not in allTime :
                    allPath.append(os.path.join(curPath,curTime))
    
    return allPath

In [5]:
def newCSVFileName(path, newName):
    return path[:-5]+newName+".csv"

In [6]:
keyList = ['id','originalPrice','discountedPrices','fastChargingPileCount','fastChargingPileNotBusyCount','slowChargingPileCount',
           'slowChargingPileNotBusyCount','chargefreeMess','currentTimePeriod','nextTimePeriod','nextTimePrice','brandStatus',
           'chargeFree','isParkingFree','parkingfeeInformation','isServiceFree','servicefreeType','servicefreeInformation',
          'fastChargingPileParameter','slowChargingPileParameter','longitude','latitude','pileDetailsList']
pileDetailsListKeywordList = ['status','statusStr','pileNo','pileType','pileModel','voltageUpperLimits','voltageLowerLimits',
                              'curRent','power']
wholeKeyWordList = keyList.copy()
wholeKeyWordList.extend(pileDetailsListKeywordList)

def exportUsableData(path):
#     print(path)
    df = pd.DataFrame(pd.read_excel(path))
#     newDF =  pd.DataFrame(columns = keyList)
    r, c = df.shape
    values = []
    for i in range(r):
        curData = []
        try:
            dataDict = json2dict(df.iloc[i,1])
        except BaseException as err:
            print(i)
        else:
            if dataDict == None:
                continue
            for k, v in dataDict.items():
                if k in keyList:
#                     print(k)
                    curData.append(v)

            detailsList = dataDict["pileDetailsList"]
            if len(detailsList) == 0:
                values.append(curData)
            else:
                for detail in detailsList:
                    curDetailData = curData.copy()
                    for k, v in detail.items():
                        if k in pileDetailsListKeywordList:
                            curDetailData.append(v)

                    values.append(curDetailData)
                        
            
    resdf = pd.DataFrame(data=values, columns = wholeKeyWordList)
    
    return resdf

In [30]:
def allExcel2csv(cityPath):
    allPath = listAllPath(cityPath)
   
    for curDayPathList in allPath:
        chargingTotalDF = pd.DataFrame(columns = ['id','status','pileNo','power','usedElec','datetime'])
        chargingStationTotalDF = pd.DataFrame(columns = ['id','number','usedStaElec','datetime'])
        for curPath in curDayPathList:
            print(curPath)
#             time = curPath.split("\\")[-1]
#             dayPath = curPath.split("\\")[0] + "\\" + curPath.split("\\")[1] + "\\" + curPath.split("\\")[2] + "\\" + curPath.split("\\")[3] + "\\" + curPath.split("\\")[4] + "\\" + curPath.split("\\")[5] + "\\" + curPath.split("\\")[6] + "\\"
#             t = time[:-5]
#             print(t)
#             T = t.split('_')
#             date = T[0] + "-" + T[1] + "-" + T[2] + " " + T[3] + ":" + T[4] + ":" + T[5]
#             pd_date = pd.to_datetime(date)   #时间戳

#             df = exportUsableData(curPath)
#             chargingDF = df[df['status'] == 3]
#             chargingDF = chargingDF.loc[:,['id','status','pileNo','power']]
#             chargingDF.insert(loc = 4, column = 'usedElec',value = chargingDF.loc[:,'power']/12)
#             chargingDF.insert(loc = 5, column = 'datetime',  value = pd_date) #充电桩每分钟统计表
#             chargingTotalDF = pd.concat([chargingTotalDF,chargingDF]) #充电桩每天统计表
# #             print(chargingTotalDF)
# #             chargingDF.to_csv(newCSVFileName(curPath, "_charging"),encoding = 'gbk')
     

#             chargingStationDF = chargingDF.id.value_counts().reset_index()
#             chargingStationDF.columns = ['id', 'number']
#             a = chargingDF['id']
#             b = chargingDF['usedElec']
#             chargingStationElecDF = pd.DataFrame({'id':a, 'usedStaElec':b})
#             usedStationElec = chargingStationElecDF.groupby('id').sum()
#             chargingStationElecDF = pd.merge(chargingStationDF, usedStationElec, on = 'id')
#             chargingStationElecDF.insert(loc = 3, column = 'datetime', value = pd_date) #充电站每分钟统计表
#             chargingStationTotalDF = pd.concat([chargingStationTotalDF,chargingStationElecDF])#充电站每天统计表
#             print(chargingStationTotalDF)
# #             chargingStationElecDF.to_csv(newCSVFileName(curPath, "_chargingStaNo_Elec"), encoding = 'gbk')


# #         chargingTotalDF.to_csv(dayPath + curPath.split("\\")[6] + '_totalCharging.csv',encoding = 'gbk')
# #         chargingStationTotalDF.to_csv(dayPath + curPath.split("\\")[6] + '_totalStaCharging.csv',encoding = 'gbk')

In [8]:
szPath = "E:\\PolyU\\gis\\Dissertation\\Data\\深圳市\\2022_7_19_0_3_24"
# szAllPath = listAllExcelPath(szPath)
# len(szAllPath)

In [16]:
def listAllPath(cityPath):
    allTime = os.listdir(cityPath)
    allPath = []
    for curTime in allTime:
        if 'xlsx' in curTime :
            if newCSVFileName(curTime, "_charging") not in allTime or newCSVFileName(curTime, "_chargingNo") not in allTime :
                allPath.append(os.path.join(cityPath,curTime))

    return allPath

In [18]:
listAllPath(szPath)

['E:\\PolyU\\gis\\Dissertation\\Data\\深圳市\\2022_7_19_0_3_24\\2022_7_18_0_11_42.xlsx',
 'E:\\PolyU\\gis\\Dissertation\\Data\\深圳市\\2022_7_19_0_3_24\\2022_7_18_0_16_29.xlsx',
 'E:\\PolyU\\gis\\Dissertation\\Data\\深圳市\\2022_7_19_0_3_24\\2022_7_18_0_21_38.xlsx',
 'E:\\PolyU\\gis\\Dissertation\\Data\\深圳市\\2022_7_19_0_3_24\\2022_7_18_0_26_44.xlsx',
 'E:\\PolyU\\gis\\Dissertation\\Data\\深圳市\\2022_7_19_0_3_24\\2022_7_18_0_31_37.xlsx',
 'E:\\PolyU\\gis\\Dissertation\\Data\\深圳市\\2022_7_19_0_3_24\\2022_7_18_0_36_38.xlsx',
 'E:\\PolyU\\gis\\Dissertation\\Data\\深圳市\\2022_7_19_0_3_24\\2022_7_18_0_41_37.xlsx',
 'E:\\PolyU\\gis\\Dissertation\\Data\\深圳市\\2022_7_19_0_3_24\\2022_7_18_0_46_37.xlsx',
 'E:\\PolyU\\gis\\Dissertation\\Data\\深圳市\\2022_7_19_0_3_24\\2022_7_18_0_51_37.xlsx',
 'E:\\PolyU\\gis\\Dissertation\\Data\\深圳市\\2022_7_19_0_3_24\\2022_7_18_0_57_32.xlsx',
 'E:\\PolyU\\gis\\Dissertation\\Data\\深圳市\\2022_7_19_0_3_24\\2022_7_18_0_6_36.xlsx',
 'E:\\PolyU\\gis\\Dissertation\\Data\\深圳市\\2022_7_19_0_

In [45]:
# chargingTotalDF = pd.DataFrame(columns = ['id','status','pileNo','power','usedElec','datetime'])
# chargingStationTotalDF = pd.DataFrame(columns = ['id','number','usedStaElec','datetime'])
# for curPath in listAllPath(szPath):
#     print(curPath)
#     time = curPath.split("\\")[-1]
#     dayPath = curPath.split("\\")[0] + "\\" + curPath.split("\\")[1] + "\\" + curPath.split("\\")[2] + "\\" + curPath.split("\\")[3] + "\\" + curPath.split("\\")[4] + "\\" + curPath.split("\\")[5] + "\\" + curPath.split("\\")[6] + "\\"
#     t = time[:-5]
#     T = t.split('_')
#     date = T[0] + "-" + T[1] + "-" + T[2] + " " + T[3] + ":" + T[4] + ":" + T[5]
#     pd_date = pd.to_datetime(date)   #时间戳
#     print(pd_date)

#     df = exportUsableData(curPath)
#     chargingDF = df[df['status'] == 3]
#     chargingDF = chargingDF.loc[:,['id','status','pileNo','power']]
#     chargingDF.insert(loc = 4, column = 'usedElec',value = chargingDF.loc[:,'power']/12)
#     chargingDF.insert(loc = 5, column = 'datetime',  value = pd_date) #充电桩每分钟统计表
#     chargingTotalDF = pd.concat([chargingTotalDF,chargingDF]) #充电桩每天统计表
# #             print(chargingTotalDF)
# #             chargingDF.to_csv(newCSVFileName(curPath, "_charging"),encoding = 'gbk')


#     chargingStationDF = chargingDF.id.value_counts().reset_index()
#     chargingStationDF.columns = ['id', 'number']
#     a = chargingDF['id']
#     b = chargingDF['usedElec']
#     chargingStationElecDF = pd.DataFrame({'id':a, 'usedStaElec':b})
#     usedStationElec = chargingStationElecDF.groupby('id').sum()
#     chargingStationElecDF = pd.merge(chargingStationDF, usedStationElec, on = 'id')
#     chargingStationElecDF.insert(loc = 3, column = 'datetime', value = pd_date) #充电站每分钟统计表
#     chargingStationTotalDF = pd.concat([chargingStationTotalDF,chargingStationElecDF])#充电站每天统计表
#     print(chargingStationTotalDF)
# #             chargingStationElecDF.to_csv(newCSVFileName(curPath, "_chargingStaNo_Elec"), encoding = 'gbk')


# #         chargingTotalDF.to_csv(dayPath + curPath.split("\\")[6] + '_totalCharging.csv',encoding = 'gbk')
print(chargingStationTotalDF)
chargingStationTotalDF.to_csv('E:\\PolyU\\gis\\Dissertation\\Data\\深圳市\\2022_7_19_0_3_24\\2022_7_19_0_3_24_totalStaCharging.csv',encoding = 'gbk')

         id number  usedStaElec            datetime
0     89925     45   391.666667 2022-07-18 00:11:42
1    108224     45   654.000000 2022-07-18 00:11:42
2     88321     43   597.500000 2022-07-18 00:11:42
3    122106     28   330.000000 2022-07-18 00:11:42
4    109500     26   260.000000 2022-07-18 00:11:42
..      ...    ...          ...                 ...
734  116034      1     0.583333 2022-07-19 00:02:39
735  116018      1     0.583333 2022-07-19 00:02:39
736  115996      1     0.583333 2022-07-19 00:02:39
737  115986      1     0.291667 2022-07-19 00:02:39
738   79851      1     7.500000 2022-07-19 00:02:39

[177674 rows x 4 columns]


In [31]:
allExcel2csv(szPath)

E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
0
_
1
1
_
4
2
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
0
_
1
6
_
2
9
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
0
_
2
1
_
3
8
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
0
_
2
6
_
4
4
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
0
_
3
1
_
3
7
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
0
_
3
6
_
3
8
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2


E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
1
4
_
2
6
_
3
6
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
1
4
_
2
_
5
0
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
1
4
_
3
1
_
3
9
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
1
4
_
3
6
_
3
9
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
1
4
_
4
1
_
3
6
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
1
4
_
4
6
_
3
7
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4


\
2
0
2
2
_
7
_
1
8
_
1
9
_
3
6
_
3
5
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
1
9
_
4
1
_
4
1
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
1
9
_
4
6
_
3
3
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
1
9
_
5
1
_
3
9
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
1
9
_
5
6
_
3
6
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
1
9
_
6
_
3
5
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
1
_
1
1
_
4
2
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a


E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
2
_
4
1
_
3
8
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
2
_
4
6
_
3
5
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
2
_
5
1
_
3
9
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
2
_
5
6
_
3
5
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
2
_
6
_
3
6
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
3
_
1
1
_
4
0
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_


2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
7
_
5
6
_
3
3
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
7
_
6
_
3
6
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
8
_
1
1
_
3
8
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
8
_
1
6
_
3
7
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
8
_
2
1
_
3
8
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
8
_
2
6
_
3
5
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a
t
i
o
n
\
D
a
t
a
\
深
圳
市
\
2
0
2
2
_
7
_
1
9
_
0
_
3
_
2
4
\
2
0
2
2
_
7
_
1
8
_
8
_
2
_
2
5
.
x
l
s
x
E
:
\
P
o
l
y
U
\
g
i
s
\
D
i
s
s
e
r
t
a


In [7]:
def listAllPath(cityPath):
    dayPath = os.listdir(cityPath)
    allDayPath = []
    for day in dayPath:
        if 'zip' not in day and 'csv' not in day:
            allDayPath.append(os.path.join(cityPath, day))
    allPath = []
    for curPath in allDayPath:
        curDayAllPath = []
        allTime = os.listdir(curPath)
        for curTime in allTime:
            if 'xlsx' in curTime :
                if newCSVFileName(curTime, "_charging") not in allTime or newCSVFileName(curTime, "_chargingNo") not in allTime :
                    curDayAllPath.append(os.path.join(curPath,curTime))
        allPath.append(curDayAllPath)
    
    return allPath

In [5]:
# type(raw)

In [6]:
# raw = df.iloc[1,1]
# raw

In [7]:
# d1 = json.loads(raw)
# d1["data"]

In [8]:
# realData = d1["data"]
# pileDetailsList = realData["pileDetailsList"]
# pileDetailsList

In [9]:
# pileDetailsList[0]["pileModel"]

In [10]:
# r, c = df.shape
# r

In [5]:
# gzPath = 'E:\PolyU\gis\Dissertation\Data\广州市'

In [12]:
# gzDay = os.listdir(gzPath)
# gzDayPath = []
# for day in gzDay:
#    if 'zip' not in day and 'csv' not in day:
#        gzDayPath.append(os.path.join(gzPath, day))
# gzDayPath

In [13]:
# gzTime = os.listdir(gzDayPath[0])
# gzTimePath = []
# for curTime in gzTime:
#    gzTimePath.append(os.path.join(gzDayPath[0], curTime))
# gzTimePath

In [14]:
# szDay = os.listdir(szPath)
# szDayPath = []
# for day in szDay:
#    if 'zip' not in day and 'csv' not in day:
#        szDayPath.append(os.path.join(szPath, day))
# szDayPath

In [15]:
# szTime = os.listdir(szDayPath[0])
# szTimePath = []
# for curTime in szTime:
#    szTimePath.append(os.path.join(szDayPath[0], curTime))
# szTimePath

In [19]:
# allPath = listAllExcelPath(gzPath)

In [20]:
# len(allPath)

In [22]:
# curpath = 'E:\\PolyU\\gis\\Dissertation\\Data\\广州市\\2022_6_19_0_2_35\\2022_6_18_1_1_16.xlsx'
# df = exportUsableData(curpath)
# df.head()
# csvname = formatCSVname(curpath)
# df.to_csv(csvname, encoding = 'gbk')

In [ ]:
# currentPath = 'E:\\PolyU\\gis\\Dissertation\\Data\\广州市\\2022_6_19_0_2_35\\2022_6_18_0_6_21.csv'
# pd.read_csv(currentPath)

In [ ]:
DataFrame = exportUsableData('E:\\PolyU\\gis\\Dissertation\\Data\\深圳市\\2022_7_22_0_3_16\\2022_7_22_0_2_37.xlsx')
chargingTotalDF = pd.DataFrame(columns = ['id','status','pileNo','power','usedElec','datetime'])
chargingStationTotalDF = pd.DataFrame(columns = ['id','number','usedStaElec','datetime'])
for curPath in curDayPathList:
print(curPath)
time = curPath.split("\\")[-1]
dayPath = curPath.split("\\")[0] + "\\" + curPath.split("\\")[1] + "\\" + curPath.split("\\")[2] + "\\" + curPath.split("\\")[3] + "\\" + curPath.split("\\")[4] + "\\" + curPath.split("\\")[5] + "\\" + curPath.split("\\")[6] + "\\"
t = time[:-5]
T = t.split('_')
date = T[0] + "-" + T[1] + "-" + T[2] + " " + T[3] + ":" + T[4] + ":" + T[5]
pd_date = pd.to_datetime(date)   #时间戳

df = exportUsableData(curPath)
chargingDF = df[df['status'] == 3]
chargingDF = chargingDF.loc[:,['id','status','pileNo','power']]
chargingDF.insert(loc = 4, column = 'usedElec',value = chargingDF.loc[:,'power']/12)
chargingDF.insert(loc = 5, column = 'datetime',  value = pd_date) #充电桩每分钟统计表
chargingTotalDF = pd.concat([chargingTotalDF,chargingDF]) #充电桩每天统计表
#             print(chargingTotalDF)
#             chargingDF.to_csv(newCSVFileName(curPath, "_charging"),encoding = 'gbk')


chargingStationDF = chargingDF.id.value_counts().reset_index()
chargingStationDF.columns = ['id', 'number']
a = chargingDF['id']
b = chargingDF['usedElec']
chargingStationElecDF = pd.DataFrame({'id':a, 'usedStaElec':b})
usedStationElec = chargingStationElecDF.groupby('id').sum()
chargingStationElecDF = pd.merge(chargingStationDF, usedStationElec, on = 'id')
chargingStationElecDF.insert(loc = 3, column = 'datetime', value = pd_date) #充电站每分钟统计表
chargingStationTotalDF = pd.concat([chargingStationTotalDF,chargingStationElecDF])#充电站每天统计表
#             print(chargingStationTotalDF)
#             chargingStationElecDF.to_csv(newCSVFileName(curPath, "_chargingStaNo_Elec"), encoding = 'gbk')


# chargingTotalDF.to_csv(dayPath + curPath.split("\\")[6] + '_totalCharging.csv',encoding = 'gbk')
chargingStationTotalDF.to_csv(dayPath + curPath.split("\\")[6] + '_totalStaCharging.csv',encoding = 'gbk')